**PROJECT | GROUP 5**

**Oguzhan Ozbek | Raghav Dheer**

**Sheridan ID #991656424 | Sheridan ID<???????>**

**PROG39051 - Machine Learning Techniques**

**Professor El Sayed Mahmoud**

**2025-04-17**

In [ ]:
#IMPORTING PYTHON LIBRARIES AND UNDERSTANDING DATA
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

df = pd.read_csv("Motorvehicleinsurancedata.csv", delimiter=';')
df.head()

In [ ]:
#DATA CLEANING
print(f'Existince of NaN values: {df.isna().any().any()}')
print(df.isna().sum())

df = df.drop('ID', axis=1)
df = df.drop('Date_lapse', axis=1)
df = df.drop('Date_last_renewal', axis=1)
df = df.drop('Date_next_renewal', axis=1)
df_clean = df.dropna()

print('\nCleaning has been completed\n')
print(f'Existince of NaN values: {df_clean.isna().any().any()}')

In [ ]:
#FEATURE EXTRACTION
df_clean['Date_birth'] = pd.to_datetime(df_clean['Date_birth'], format='%d/%m/%Y', errors='coerce')
today = pd.to_datetime("today")
df_clean['Age'] = (today - df_clean['Date_birth']).dt.days // 365

df_clean['Date_start_contract'] = pd.to_datetime(df_clean['Date_start_contract'], format='%d/%m/%Y', errors='coerce')
today = pd.to_datetime("today")
df_clean['Length_contract'] = (today - df_clean['Date_start_contract']).dt.days // 365

df_clean['Date_driving_licence'] = pd.to_datetime(df_clean['Date_driving_licence'], format='%d/%m/%Y', errors='coerce')
today = pd.to_datetime("today")
df_clean['Length_licence'] = (today - df_clean['Date_driving_licence']).dt.days // 365

df_clean = df_clean.drop('Date_birth', axis=1)
df_clean = df_clean.drop('Date_start_contract', axis=1)
df_clean = df_clean.drop('Date_driving_licence', axis=1)

df_clean.info()